In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class CrowdDataset(Dataset):
    def __init__(self, image_paths, ground_truth_paths, transform=None, augment=False, patch_size=256):
        self.image_paths = image_paths
        self.ground_truth_paths = ground_truth_paths
        self.transform = transform
        self.augment = augment
        self.patch_size = patch_size
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Load ground truth from .mat file
        gt_path = self.ground_truth_paths[idx]
        try:
            mat_data = scipy.io.loadmat(gt_path)
            # Extract point coordinates from image_info[0,0][0,0][0]
            points = mat_data['image_info'][0, 0][0, 0][0]
            # Convert to list of [x, y] coordinates if it's a valid array
            if points.size > 0 and len(points.shape) == 2 and points.shape[1] == 2:
                points = points.tolist()
            else:
                points = []
        except Exception as e:
            print(f"Error loading ground truth file {gt_path}: {e}")
            points = []
        
        # Generate density map
        density_map = self.generate_density_map(img.shape[:2], points)
        
        # Apply augmentation if specified
        if self.augment:
            img, density_map = self.apply_augmentation(img, density_map)
        
        # Resize to patch size
        img = cv2.resize(img, (self.patch_size, self.patch_size))
        density_map = cv2.resize(density_map, (self.patch_size, self.patch_size))
        
        # Convert to tensor
        img = torch.from_numpy(img.transpose(2, 0, 1)).float() / 255.0
        density_map = torch.from_numpy(density_map).float().unsqueeze(0)
        
        # Apply transforms if any
        if self.transform:
            img = self.transform(img)
            
        return img, density_map
    
    def generate_density_map(self, img_shape, points):
        """Generate density map from point annotations using adaptive Gaussian kernels"""
        h, w = img_shape
        density_map = np.zeros((h, w), dtype=np.float32)
        
        if len(points) == 0:
            return density_map
        
        # Convert points to numpy array
        points = np.array(points)
        
        # For each point, calculate adaptive sigma based on k-nearest neighbors
        for i, point in enumerate(points):
            x, y = int(point[0]), int(point[1])
            
            # Skip if point is outside image bounds
            if x < 0 or x >= w or y < 0 or y >= h:
                continue
            
            # Calculate distance to k nearest neighbors (k=3)
            if len(points) > 1:
                distances = np.sqrt(np.sum((points - point) ** 2, axis=1))
                distances = np.sort(distances)[1:]  # Exclude self (distance=0)
                
                # Use average of 3 nearest neighbors or all if less than 3
                k = min(3, len(distances))
                avg_distance = np.mean(distances[:k])
                
                # Adaptive sigma based on local density
                sigma = max(1.0, avg_distance / 3.0)
            else:
                sigma = 4.0  # Default sigma for single point
            
            # Create Gaussian kernel
            kernel_size = int(6 * sigma)
            if kernel_size % 2 == 0:
                kernel_size += 1
            
            # Generate 2D Gaussian
            gaussian = self.gaussian_2d(kernel_size, sigma)
            
            # Add to density map
            x_start = max(0, x - kernel_size // 2)
            x_end = min(w, x + kernel_size // 2 + 1)
            y_start = max(0, y - kernel_size // 2)
            y_end = min(h, y + kernel_size // 2 + 1)
            
            # Adjust Gaussian bounds
            g_x_start = max(0, kernel_size // 2 - x)
            g_x_end = g_x_start + (x_end - x_start)
            g_y_start = max(0, kernel_size // 2 - y)
            g_y_end = g_y_start + (y_end - y_start)
            
            density_map[y_start:y_end, x_start:x_end] += gaussian[g_y_start:g_y_end, g_x_start:g_x_end]
        
        return density_map
    
    def gaussian_2d(self, kernel_size, sigma):
        """Generate 2D Gaussian kernel"""
        kernel = np.zeros((kernel_size, kernel_size))
        center = kernel_size // 2
        
        for i in range(kernel_size):
            for j in range(kernel_size):
                x, y = i - center, j - center
                kernel[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma**2))
        
        return kernel / np.sum(kernel)
    
    def apply_augmentation(self, img, density_map):
        """Apply data augmentation"""
        # Random horizontal flip
        if random.random() > 0.5:
            img = cv2.flip(img, 1)
            density_map = cv2.flip(density_map, 1)
        
        # Random brightness adjustment
        if random.random() > 0.5:
            brightness_factor = random.uniform(0.8, 1.2)
            img = np.clip(img * brightness_factor, 0, 255).astype(np.uint8)
        
        # Random noise
        if random.random() > 0.7:
            noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
            img = np.clip(img + noise, 0, 255).astype(np.uint8)
        
        return img, density_map
class PReLU(nn.Module):
    """Parametric ReLU activation function"""
    def __init__(self, num_parameters=1):
        super(PReLU, self).__init__()
        self.num_parameters = num_parameters
        self.weight = nn.Parameter(torch.Tensor(num_parameters).fill_(0.25))
    
    def forward(self, x):
        return F.prelu(x, self.weight)